In [20]:
#Extract Payments per hour from an external source:
#In this case, a text file (Could be a table from a database like MySQL or PostgreSQL)
payment_structure=dict()
fileName='payment_table.txt'
with open(fileName) as f:
    lines = f.readlines()
for line in lines:
    [day,str_start_time,str_end_time,str_price]=line.strip().split(' ')
    start_time=int(str_start_time.split(':')[0])
    end_time=int(str_end_time.split(':')[0])
    price=int(str_price)
    #print(day+','+str_start_time+','+str_end_time+','+str_price)
    if not day in payment_structure:
        payment_structure[day]=[]
    payment_structure[day].append([start_time,end_time,price])
print(payment_structure)

{'MO': [[0, 9, 25], [9, 18, 15], [18, 0, 20]], 'TU': [[0, 9, 25], [9, 18, 15], [18, 0, 20]], 'WE': [[0, 9, 25], [9, 18, 15], [18, 0, 20]], 'TH': [[0, 9, 25], [9, 18, 15], [18, 0, 20]], 'FR': [[0, 9, 25], [9, 18, 15], [18, 0, 20]], 'SA': [[0, 9, 30], [9, 18, 20], [18, 0, 25]], 'SU': [[0, 9, 30], [9, 18, 20], [18, 0, 25]]}


In [45]:
fileName='emp2.txt'
final_payment=0
with open(fileName) as f:
    lines = f.readlines()
for line in lines:
    #to-do: check incorrect format of text file
    #to-do: make this work for several lines: Compose complete sheet of employees and salaries
    [name,data]=line.split('=')
    #print(name)
    items=data.split(',')
    for item in items:
        day_of_week=item[0:2]
        time_interval=item[2:]
        [str_start_time,str_end_time]=time_interval.split('-')
        #print(name+', '+day_of_week+', '+str_start_time+' to '+str_end_time)
        #to-do: check if end is greater than start
        #to-do: check minutes into hours (what if half an hour? or a couple of minutes?)
        #to-do: check intervals that belong to more than one slot
        start_time=int(str_start_time.split(':')[0])
        end_time=int(str_end_time.split(':')[0])
        #to-do: Check if there is an array of values for the corresponding day_of_week
        #Check if there exists an interval containing the time span
        time_spans=payment_structure[day_of_week]
        print(day_of_week)
        #print(time_spans)
        for span in time_spans:
            print('span: '+str(span))
            span_start=span[0]
            span_end=span[1]
            #In case we have a span like 18:00-00:00 or 21:00-04:00
            if (span_end<span_start):
                span_end+=24
            if start_time>=span_start and start_time<span_end:
                hours=end_time-start_time
                payment_per_hour=span[2]
                partial_amount=hours*payment_per_hour
                final_payment+=partial_amount
                print('span start: '+str(span[0]))
                print('span end: '+str(span[1]))
                print('item start: '+str(start_time))
                print('item end: '+str(end_time))
                print('hours: '+str(hours))
                print('payment per hour: U$s'+str(span[2]))
                print('partial payment: U$s'+str(partial_amount))
                break
print('The amount to pay {} is: {} USD'.format(name,final_payment))

MO
span: [0, 9, 25]
span: [9, 18, 15]
span start: 9
span end: 18
item start: 10
item end: 12
hours: 2
payment per hour: U$s15
partial payment: U$s30
TH
span: [0, 9, 25]
span: [9, 18, 15]
span start: 9
span end: 18
item start: 12
item end: 14
hours: 2
payment per hour: U$s15
partial payment: U$s30
SU
span: [0, 9, 30]
span: [9, 18, 20]
span: [18, 0, 25]
span start: 18
span end: 0
item start: 20
item end: 21
hours: 1
payment per hour: U$s25
partial payment: U$s25
The amount to pay ASTRID is: 85 USD


In [61]:
class Employee:
    def __init__(self,name):
        self.name=name
    
    def set_payment_structure(self,payment_structure):
        self.payment_structure=payment_structure
    
    def calculate_payment(self,str_hours_report):
        final_payment=0
        items=str_hours_report.split(',')
        #to-do: test malformed string
        for item in items:
            day_of_week=item[0:2]
            time_interval=item[2:]
            final_payment+=self.payment_structure.partial_amount(day_of_week,time_interval)
        return final_payment
                
class PaymentStructure:
    def extract_from_string(self,str_payment_structure):
        self.compose_payment_structure(str_payment_structure.split('\n'))
    
    def extract_from_file(self,filePaymentStructure):
        #to-do: check incorrect format of text file
        with open(filePaymentStructure) as f:
            lines = f.readlines()
        self.compose_payment_structure(lines)
        
    def compose_payment_structure(self,lines):
        self.payment_structure=dict()
        for line in lines:
            [day,str_start_time,str_end_time,str_price]=line.strip().split(' ')
            start_time=int(str_start_time.split(':')[0])
            end_time=int(str_end_time.split(':')[0])
            price=int(str_price)
            #print(day+','+str_start_time+','+str_end_time+','+str_price)
            if not day in self.payment_structure:
                self.payment_structure[day]=[]
            self.payment_structure[day].append([start_time,end_time,price])
        
    def partial_amount(self,day_of_week,time_interval):
        [str_start_time,str_end_time]=time_interval.split('-')
        #to-do: check if end is greater than start
        #to-do: check minutes into hours (what if half an hour? or a couple of minutes?)
        start_time=int(str_start_time.split(':')[0])
        end_time=int(str_end_time.split(':')[0])
        #Check if there exists an interval containing the time span
        for time_span in self.payment_structure[day_of_week]:
            span_start=time_span[0]
            span_end=time_span[1]
            #In case we have a span like 18:00-00:00 or 21:00-04:00
            if (span_end<span_start):
                span_end+=24
            if start_time>=span_start and start_time<span_end:
                #to-do: check intervals that belong to more than one slot
                hours=end_time-start_time
                payment_per_hour=time_span[2]
                return hours*payment_per_hour

            
#to-do: check incorrect format of text file
#to-do: make this work for several lines: Compose complete sheet of employees and salaries            
hours_report_file='data/emp2.txt'
payment_str_file='data/payment_table.txt'

with open(hours_report_file) as f:
    lines = f.readlines()
if len(lines)!=1:
    raise TypeError
[name,hours_report]=lines[0].split('=')
emp=Employee(name)
pay_str=PaymentStructure()
pay_str.extract_from_file(payment_str_file)
emp.set_payment_structure(pay_str)
final_payment=emp.calculate_payment(hours_report)
print('The amount to pay {} is: {} USD'.format(emp.name,final_payment))

The amount to pay ASTRID is: 85 USD
